# Setup Environment


In [ ]:
!pip install ultralytics


In [ ]:
!pip install deep-sort-realtime

In [ ]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab.patches import cv2_imshow


# Load the YOLO Model

In [ ]:
# Load YOLOv8 model
model = YOLO('yolov8n.pt')  # or any other yolov8 model

# Initialize DeepSort
tracker = DeepSort(max_age=30)

# Open video file
cap = cv2.VideoCapture('/content/Video.mp4')

# Check if video opened successfully
if not cap.isOpened():
    print("Error opening video file")
    exit()

#  Traffic Density and Estimated Waiting Time Calculation


In [ ]:
import cv2

# Initialize YOLOv8 model and DeepSort tracker (ensure they are properly loaded)
# Example: model = YOLOv8() and tracker = DeepSort()

# Counting variables
vehicle_count = 0
tracking_objects = {}

# Initialize waiting time and density variables
traffic_density = 'Low'
waiting_time = 5  # Default waiting time for low density

# Start video capture
cap = cv2.VideoCapture('/content/Video.mp4')  # Replace with the path to your video

# Get the width, height, and frames per second (FPS) of the video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object to save the output video
output_file = 'output_video_2.avi'  # Output video file name
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can use 'mp4v' for .mp4 files
out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("End of video or error reading frame.")
        break  # Exit the loop if the end of the video is reached

    # YOLOv8 object detection on the current frame
    results = model(frame)
    detections = []

    # Process YOLOv8 detections
    for result in results[0].boxes:
        bbox = result.xyxy[0].cpu().numpy()  # Bounding box in (xmin, ymin, xmax, ymax) format
        conf = result.conf[0].cpu().numpy()  # Confidence score
        label = int(result.cls[0].cpu().numpy())  # Class label

        # Vehicle class labels (e.g., car, truck, bus, motorcycle)
        if label in [2, 3, 5, 7]:  # Adjust based on YOLO model's class indices for vehicles
            detections.append((bbox, conf, label))

    # DeepSort tracking
    tracks = tracker.update_tracks(detections, frame=frame)

    # Reset vehicle count per frame
    current_vehicle_count = 0

    # Process the tracking results
    for track in tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue

        track_id = track.track_id
        bbox = track.to_tlbr()  # Get bounding box for tracking
        x1, y1, x2, y2 = [int(i) for i in bbox]

        # Count the current vehicle
        current_vehicle_count += 1

        # Draw bounding box and tracking ID on the frame
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Determine traffic density and waiting time based on the current vehicle count
    if current_vehicle_count < 10:
        traffic_density = 'Low'
        waiting_time = 5  # Low density waiting time
    elif 10 <= current_vehicle_count <= 20:
        traffic_density = 'Medium'
        waiting_time = 10  # Medium density waiting time
    else:
        traffic_density = 'High'
        waiting_time = 20  # High density waiting time

    # Display vehicle count, traffic density, and waiting time on the frame
    cv2.putText(frame, f'Vehicles: {current_vehicle_count}', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.putText(frame, f'Traffic Density: {traffic_density}', (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.putText(frame, f'Waiting Time: {waiting_time} min', (20, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Write the processed frame to the output video
    out.write(frame)


    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and writer, and close windows
cap.release()
out.release()
cv2.destroyAllWindows()


Streaming output truncated to the last 5000 lines.
Speed: 2.6ms preprocess, 6.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 3 trucks, 6.8ms
Speed: 2.7ms preprocess, 6.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 cars, 3 trucks, 7.2ms
Speed: 2.6ms preprocess, 7.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 2 trucks, 7.2ms
Speed: 2.6ms preprocess, 7.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 cars, 1 bus, 2 trucks, 7.0ms
Speed: 2.5ms preprocess, 7.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 3 trucks, 7.0ms
Speed: 2.7ms preprocess, 7.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 cars, 1 bus, 2 trucks, 7.0ms
Speed: 2.4ms preprocess, 7.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 25 cars, 1 truck, 7.8ms
Speed

# save the model

In [ ]:
model.save('vehicle_count.pt')